In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split 
import time
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
import pickle
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier   
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings("ignore")


In [3]:
def rfeFeature(indep_X, dep_Y, n):
    rfelist = []
    
    # Models to apply RFE with
    log_model = LogisticRegression(solver='lbfgs')
    rf_model = RandomForestClassifier(n_estimators=10, criterion='entropy', random_state=0)
    dt_model = DecisionTreeClassifier(criterion='gini', max_features='sqrt', splitter='best', random_state=0)
    svc_model = LinearSVC(random_state=0)  # Use LinearSVC for feature selection
    
    rfemodellist = [log_model, svc_model, rf_model, dt_model]

    # Iterate through models and apply RFE
    for model in rfemodellist:
        # Create an RFE model and select n features
        rfe = RFE(estimator=model, n_features_to_select=n)
        rfe = rfe.fit(indep_X, dep_Y)
        
        # Get the selected features
        selected_features = indep_X.columns[rfe.support_]
        
        print("Selected features for {model.class.name}: {selected_features}")
        rfelist.append(selected_features)
    
    return rfelist

In [4]:
def split_scalar(indep_X,dep_Y):
        X_train, X_test, y_train, y_test = train_test_split(indep_X, dep_Y, test_size = 0.25, random_state = 0)
        #X_train, X_test, y_train, y_test = train_test_split(indep_X,dep_Y, test_size = 0.25, random_state = 0)
        
        #Feature Scaling
        #from sklearn.preprocessing import StandardScaler
        sc = StandardScaler()
        X_train = sc.fit_transform(X_train)
        X_test = sc.transform(X_test)
        
        return X_train, X_test, y_train, y_test
    

In [5]:
def cm_prediction(classifier,X_test):
     y_pred = classifier.predict(X_test)
     # Making the Confusion Matrix
     from sklearn.metrics import confusion_matrix
     cm = confusion_matrix(y_test, y_pred)
     from sklearn.metrics import accuracy_score 
     from sklearn.metrics import classification_report 
    #from sklearn.metrics import confusion_matrix
    #cm = confusion_matrix(y_test, y_pred)
     Accuracy=accuracy_score(y_test, y_pred )
     report=classification_report(y_test, y_pred)
     return  classifier,Accuracy,report,X_test,y_test,cm

In [6]:
def logistic(X_train,y_train,X_test):       
        # Fitting K-NN to the Training set
        from sklearn.linear_model import LogisticRegression
        classifier = LogisticRegression(random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm      
    

In [7]:
def svm_rbf(X_train,y_train,X_test):
        from sklearn.svm import SVC
        classifier = SVC(kernel = 'rbf', random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm

In [8]:
def Navie(X_train,y_train,X_test):       
        # Fitting K-NN to the Training set
        from sklearn.naive_bayes import GaussianNB
        classifier = GaussianNB()
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm         
    

In [9]:
def knn(X_train,y_train,X_test):
        # Fitting K-NN to the Training set
        from sklearn.neighbors import KNeighborsClassifier
        classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm


In [10]:
def Decision(X_train,y_train,X_test):
        # Fitting K-NN to the Training set
        from sklearn.tree import DecisionTreeClassifier
        classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm    

In [11]:
def random(X_train,y_train,X_test):
        # Fitting K-NN to the Training set
        from sklearn.ensemble import RandomForestClassifier
        classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm
    

In [12]:
def rfe_classification(acclog,accrbf,accknn,accnav,accdes,accrf): 
    
    rfedataframe=pd.DataFrame(index=['Logistic','SVC','Random','DecisionTree','KNN'],columns=['Logistic','rbf',
                                                                                        'KNN','Navie','Decision','Random'])
    for number, idex in enumerate(rfedataframe.index):
        # Use .loc to assign values based on row index and column name
        rfedataframe.loc[idex, 'Logistic'] = acclog[number]
        rfedataframe.loc[idex, 'SVC'] = accrbf[number]
        rfedataframe.loc[idex, 'KNN'] = accknn[number]
        rfedataframe.loc[idex, 'Navie'] = accnav[number]
        rfedataframe.loc[idex, 'Decision'] = accdes[number]
        rfedataframe.loc[idex, 'Random'] = accrf[number]
    return rfedataframe

In [13]:
dataset1=pd.read_csv("preprocessed_house_rent.csv",index_col=None)
df2=dataset1
df2 = pd.get_dummies(df2, drop_first=True,dtype=int)
indep_X=df2.drop('Total Floors',axis=1)
dep_Y=df2['Total Floors']

In [14]:
df2

,BHK,Rent,Size,Area Type,Furnishing Status,Tenant Preferred,Bathroom,City_Chennai,City_Delhi,City_Hyderabad,City_Kolkata,City_Mumbai,Floor Number,Total Floors
0,2,10000,1100,2,2,1,2,0,0,0,1,0,0,2
1,2,20000,800,2,1,1,1,0,0,0,1,0,1,3
2,2,17000,1000,2,1,1,1,0,0,0,1,0,1,3
3,2,10000,800,2,2,1,1,0,0,0,1,0,1,2
4,2,7500,850,1,2,0,1,0,0,0,1,0,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4741,2,15000,1000,1,1,1,2,0,0,1,0,0,3,5
4742,3,29000,2000,2,1,1,3,0,0,1,0,0,1,4
4743,3,35000,1750,1,1,1,3,0,0,1,0,0,3,5
4744,3,45000,1500,1,1,2,2,0,0,1,0,0,23,34


In [15]:
rfelist=rfeFeature(indep_X,dep_Y,3)       
acclog=[]
accrbf=[]
accknn=[]
accnav=[]
accdes=[]
accrf=[]


    

Selected features for {model.class.name}: {selected_features}
Selected features for {model.class.name}: {selected_features}
Selected features for {model.class.name}: {selected_features}
Selected features for {model.class.name}: {selected_features}
